## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [14]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [15]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [16]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [17]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [18]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [19]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [53]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [54]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해  {language} 로 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [59]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리" , "language": "한국어"}

In [60]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**  
먼저 인공지능이 배우기 위해 많은 예시(데이터)를 모아요. 예를 들어, 사진을 통해 고양이와 강아지를 구별하는 인공지능을 만든다면, 수많은 고양이 사진과 강아지 사진이 필요하겠죠.

2. **모델 설계**  
그 다음, 인공지능이 어떤 방식으로 정보를 처리할지 정하는 '모델'을 만듭니다. 이 모델은 사람의 뇌와 비슷하게 여러 층으로 이루어진 '신경망'이라는 구조를 갖고 있어요.

3. **학습 과정**  
이제 모델이 데이터를 보고 배우기 시작해요. 모델은 사진을 보고 '이건 고양이야' 또는 '이건 강아지야'라고 예측을 합니다. 그런데 처음에는 맞거나 틀릴 수 있어요.

4. **오차 계산과 수정**  
모델이 예측한 결과와 실제 정답(레이블)과 비교해서 얼마나 틀렸는지 '오차'를 계산해요. 그리고 이 오차를 줄이기 위해 모델 내부의 '가중치'라는 값을 조금씩 조정합니다. 이 과정을 반복하면서 모델은 점점 더 정확하게 예측하게 돼요.

5. **반복 학습**  
이 과정을 여러 번 반복하면서 모델은 데이터의 특징을 잘 파악하게 되고, 새로운 데이터에 대해서도 올바르게 판단할 수 있게 됩니다.

요약하자면, 인공지능은 많은 데이터를 보고, 예측을 하고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정입니다. 이렇게 학습을 통해 다양한 문제를 해결할 수 있게 되는 거예요!

### 출력파서(Output Parser)


In [50]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [55]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [57]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리" , "language": "영어"}
chain.invoke(input)

'Sure! Here\'s a simple explanation of how artificial intelligence (AI) models learn:\n\nArtificial intelligence models learn by looking at lots of examples. Imagine teaching a child to recognize animals. You show pictures of cats and dogs, and tell them which is which. Over time, the child notices patterns—cats have pointy ears, dogs have floppy ears, etc. \n\nSimilarly, AI models are trained with many examples. They analyze the data and find patterns or rules. For example, if the model is learning to recognize pictures of cats, it looks at many images labeled "cat" and learns what features are common in those images.\n\nDuring training, the model makes guesses about new data. If it guesses wrong, it adjusts itself a little to do better next time. This process is called "learning." After many rounds of guessing and adjusting, the model gets better at making accurate predictions or recognizing patterns.\n\nIn short, AI models learn by practicing with lots of data, recognizing patterns,

In [58]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

Sure! Here's a simple explanation of how artificial intelligence (AI) models learn:

Artificial intelligence models learn by looking at lots of examples. Imagine teaching a child to recognize animals. You show pictures of cats and dogs, and tell them which is which. Over time, the child notices patterns—cats have pointy ears, dogs have floppy ears, etc. 

Similarly, AI models are trained with many data examples. They analyze these examples to find patterns and relationships. For example, if you want an AI to recognize pictures of cats, you give it many images labeled "cat." The model then learns what features are common in those images.

Once trained, the AI can look at new, unseen pictures and decide if they are cats or not, based on what it has learned. This process of learning from data and finding patterns is how AI models get smarter over time!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [67]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [68]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [69]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
"Hello, I would like to order a meal, please."  
Could I see the menu?  
I'll have the chicken pasta, and a glass of water, please.  

- 한글 해석:  
"안녕하세요, 음식을 주문하고 싶어요."  
메뉴를 볼 수 있을까요?  
치킨 파스타로 할게요, 그리고 물 한 잔 주세요.


In [70]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello! I’d like to order a meal, please."  
"Hi! Can I have the menu, please?"  
"Could I get the special today?"  
"I’ll have the chicken pasta, please."  
"Can I also get a glass of water?"  

- 한글 해석:  
"안녕하세요! 음식을 주문하고 싶어요."  
"안녕하세요! 메뉴 좀 주세요."  
"오늘의 스페셜을 주문할 수 있을까요?"  
"치킨 파스타로 할게요."  
"물 한 잔도 같이 주세요."

In [71]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello! I’d like to order a large pepperoni pizza and a side of garlic knots, please."  
"Sure! Would you like to add a drink or any other sides?"  
  
- 한글 해석:  
“안녕하세요! 큰 페퍼로니 피자와 갈릭 노트(마늘빵)를 주문하고 싶은데요.”  
“네, 알겠습니다! 음료수나 다른 사이드도 추가하시겠어요?”